Environment and Import Preparation

In [2]:
!pip install torch torchvision torchaudio -q
!pip install torch-geometric -q
!pip install dgl -q  # generic DGL (CPU/GPU autodetect)
!pip install torchmetrics==1.4.0.post0 scikit-learn pandas numpy tqdm geopy haversine -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 22.5 MB/s eta 0:00:00


In [3]:
import os, json, math, random, gc, time
from dataclasses import dataclass
from typing import Dict, Tuple, List, Optional

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torch_geometric.data import HeteroData
from torch_geometric.utils import to_undirected, coalesce
from torch_geometric.nn import HGTConv, SAGEConv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from haversine import haversine

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED);
if DEVICE.type == 'cuda':
    torch.cuda.manual_seed_all(SEED)

print("Device:", DEVICE)

Device: cuda


JSON processing

In [10]:
import json

with open("filter_all_t.json", "r") as f:
    data = json.load(f)

print(type(data))
print(list(data.keys())[:10] if isinstance(data, dict) else data[0])

<class 'dict'>
['train', 'val', 'test']


In [11]:
import json
import pandas as pd

# Load the JSON file
with open("filter_all_t.json", "r") as f:
    data = json.load(f)

# Loop through each dataset and save separately
for split_name, records in data.items():
    df = pd.DataFrame(records)
    csv_name = f"{split_name}.csv"
    df.to_csv(csv_name, index=False)
    print(f"✅ Saved {csv_name} with {len(df)} rows.")

✅ Saved train.csv with 87013 rows.
✅ Saved val.csv with 10860 rows.
✅ Saved test.csv with 11015 rows.


Data loader for train/test/val

In [9]:
!fusermount -u /content/drive
!rm -rf /content/drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

fusermount: failed to unmount /content/drive: Invalid argument
Mounted at /content/drive


In [10]:
# Loader for train/test/val sets

import os, time
import numpy as np
import pandas as pd

# Paths to your three CSVs
DATA_DIR = "/content/drive/MyDrive/CS224W/Data"
os.makedirs(DATA_DIR, exist_ok=True)

TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
VAL_CSV   = os.path.join(DATA_DIR, "val.csv")
TEST_CSV  = os.path.join(DATA_DIR, "test.csv")

MIN_USER_INTERACTIONS = 2
MIN_ITEM_INTERACTIONS = 2

def _ensure_required_cols(df: pd.DataFrame, split_name: str) -> pd.DataFrame:
    df = df.copy()

    # Map your headings to the schema the pipeline expects
    # business_id -> item_id (keep the original column intact for reference)
    if "business_id" in df.columns and "item_id" not in df.columns:
        df["item_id"] = df["business_id"].astype(str)

    # Ensure user_id is string
    if "user_id" in df.columns:
        df["user_id"] = df["user_id"].astype(str)
    else:
        raise ValueError(f"{split_name} is missing 'user_id' column.")

    # Ensure rating exists and is numeric
    if "rating" not in df.columns:
        df["rating"] = 5.0
    df["rating"] = pd.to_numeric(df["rating"], errors="coerce").fillna(5.0)

    # Timestamp (not present in CSV) — create a stable dummy ts
    # Use the row index to create increasing integers (works for sorting)
    if "ts" not in df.columns:
        df["ts"] = np.arange(len(df), dtype=np.int64)

    # Placeholders for metadata we don't have yet
    for c in ["user_lat", "user_lon", "item_lat", "item_lon", "price"]:
        if c not in df.columns:
            df[c] = np.nan

    # Categories placeholder (list-like)
    if "categories" not in df.columns:
        df["categories"] = [[] for _ in range(len(df))]

    # Tag the split (handy later)
    df["split"] = split_name
    return df

def load_google_restaurants() -> pd.DataFrame:
    """Loads train/val/test CSVs (with headers: business_id, user_id, rating, review_text, pics, history_reviews),
    standardizes to the expected schema, and returns a single combined DataFrame with a 'split' column.
    """
    existing = [p for p in [TRAIN_CSV, VAL_CSV, TEST_CSV] if os.path.exists(p)]
    if not existing:
        raise FileNotFoundError("Couldn't find train.csv, val.csv, or test.csv in DATA_DIR")

    dfs = []
    for path in existing:
        split_name = os.path.splitext(os.path.basename(path))[0]  # "train"/"val"/"test"
        raw = pd.read_csv(path)
        std = _ensure_required_cols(raw, split_name)
        dfs.append(std)

    df_all = pd.concat(dfs, ignore_index=True)
    print(f"Loaded {len(df_all)} rows from {len(dfs)} files: {[os.path.basename(p) for p in existing]}")
    return df_all

df = load_google_restaurants()
print(df.head(), "\n", df.shape)


Loaded 108888 rows from 3 files: ['train.csv', 'val.csv', 'test.csv']
                business_id                user_id  rating  \
0  60567465d335d0abfb415b26  101074926318992653684       4   
1  6050fa9f5b4ccec8d5cae994  117065749986299237881       5   
2  604be10877e81aaed3cc9a1e  106700937793048450809       4   
3  60411e017cd8bf130362365a  101643045857250355161       5   
4  604139dd7cd8bf1303624208  109802745326785766951       4   

                                         review_text  \
0  The tang of the tomato sauce is outstanding. A...   
1              Chicken and waffles were really good!   
2  The appetizer of colossal shrimp was very good...   
3  The fish tacos here  omg! The salad was great ...   
4  Ribs are great, as are the mac and cheese, fri...   

                                                pics  \
0  ['AF1QipM-2IRmvitARbcJr7deWfe5hyVBg_ArPMQSYvq0...   
1   ['AF1QipMpfxIZUT_aymQ3qPGO-QgGYzxbtLZGmHufAp2s']   
2  ['AF1QipMNnqM5X9sSyZ9pXRZ1jvrURHN9bZhGdzuEXoP8...

Filter/encode/split and build the graph with train edges

In [11]:
import math
import torch
from sklearn.preprocessing import LabelEncoder
from torch_geometric.data import HeteroData
from torch_geometric.utils import to_undirected, coalesce

DEVICE = "cpu"  # change to "cuda" if you have a GPU

# ---------- Filtering ----------
def filter_min_interactions(df, umin=MIN_USER_INTERACTIONS, imin=MIN_ITEM_INTERACTIONS):
    """Filter out users/items globally with < umin/imin interactions."""
    grouped_u = df.groupby("user_id").size()
    keep_users = set(grouped_u[grouped_u >= umin].index)
    grouped_i = df.groupby("item_id").size()
    keep_items = set(grouped_i[grouped_i >= imin].index)
    out = df[df.user_id.isin(keep_users) & df.item_id.isin(keep_items)].copy()
    return out

df_all = filter_min_interactions(df)
print("After filtering:", df_all.shape)

# ---------- Global encoding ----------
u_enc = LabelEncoder().fit(df_all["user_id"])
i_enc = LabelEncoder().fit(df_all["item_id"])

def encode_df(subdf: pd.DataFrame) -> pd.DataFrame:
    subdf = subdf.copy()
    # Only keep rows that survived global filtering
    subdf = subdf[subdf["user_id"].isin(u_enc.classes_) & subdf["item_id"].isin(i_enc.classes_)]
    subdf["u"] = u_enc.transform(subdf["user_id"])
    subdf["i"] = i_enc.transform(subdf["item_id"])
    return subdf

train_df = encode_df(df_all[df_all["split"] == "train"])
val_df   = encode_df(df_all[df_all["split"] == "val"])
test_df  = encode_df(df_all[df_all["split"] == "test"])

num_users = len(u_enc.classes_)
num_items = len(i_enc.classes_)
print(f"num_users={num_users}, num_items={num_items}")

# ---------- Build hetero graph (train edges only) ----------
data = HeteroData()
data["user"].num_nodes = num_users
data["item"].num_nodes = num_items

# user-item edges from train
ui_src = torch.tensor(train_df["u"].values, dtype=torch.long)
ui_dst = torch.tensor(train_df["i"].values, dtype=torch.long)
edge_index = torch.stack([ui_src, ui_dst], dim=0)
data["user", "rates", "item"].edge_index = edge_index
data["item", "rev_by", "user"].edge_index = edge_index.flip(0)

# ---------- Item features (metadata not yet available) ----------
# Keep the expected 3-dim feature shape [item_lat, item_lon, price], all zeros.
# This matches downstream code expecting a (num_items, 3) tensor.
item_x = torch.zeros((num_items, 3), dtype=torch.float)
data["item"].x = item_x

# ---------- Item-item edges ----------
# Co-review edges from train: connect items co-rated by the same user at least twice.
co_counts = {}
for u, grp in train_df.groupby("u"):
    items = grp["i"].tolist()
    for a in items:
        for b in items:
            if a >= b:
                continue
            co_counts[(a, b)] = co_counts.get((a, b), 0) + 1

pairs = [(a, b) for (a, b), c in co_counts.items() if c >= 2]
if len(pairs) > 0:
    ii_src = [a for a, b in pairs]
    ii_dst = [b for a, b in pairs]
    ii_edge = torch.tensor([ii_src, ii_dst], dtype=torch.long)
    ii_edge = to_undirected(ii_edge)
    data["item", "similar", "item"].edge_index = coalesce(ii_edge, num_nodes=num_items)
else:
    # If no pairs meet the threshold, create an empty edge_index
    data["item", "similar", "item"].edge_index = torch.empty((2,0), dtype=torch.long)

data = data.to(DEVICE)
print(data)

After filtering: (96003, 15)
num_users=36364, num_items=17946
HeteroData(
  user={ num_nodes=36364 },
  item={
    num_nodes=17946,
    x=[17946, 3],
  },
  (user, rates, item)={ edge_index=[2, 76769] },
  (item, rev_by, user)={ edge_index=[2, 76769] },
  (item, similar, item)={ edge_index=[2, 3652] }
)


Sampling, Metrics, and Utilities

In [12]:
import random
import math
import torch
import numpy as np
from typing import Optional, Dict, List, Tuple

# ---------- BPR Sampler ----------
def bpr_triplet_sampler(train_df: pd.DataFrame, num_items: int,
                        radius_km: Optional[float]=None,
                        item_latlon: Optional[List[Tuple[float,float]]]=None,
                        user_pos_map: Optional[Dict[int,set]]=None,
                        user_home_latlon: Optional[Dict[int,Tuple[float,float]]]=None,
                        batch_size: int = 2048):
    """
    Yields batches of (u, pos_i, neg_j) for BPR.
    Works even if geographic metadata is missing.
    """
    # Build user -> positive item map
    if user_pos_map is None:
        user_pos_map = {u: set(g["i"].values.tolist()) for u, g in train_df.groupby("u")}
    users = list(user_pos_map.keys())

    # Precompute safe item_latlon array if not provided
    if item_latlon is None or len(item_latlon) != num_items:
        item_latlon = [(np.nan, np.nan)] * num_items

    while True:
        uu, ii, jj = [], [], []
        for _ in range(batch_size):
            u = random.choice(users)
            pos_i = random.choice(list(user_pos_map[u]))

            # Negative sampling
            if radius_km is not None and item_latlon is not None:
                neg_j = None
                # Determine user's location center (home or positive item)
                center = None
                if user_home_latlon and u in user_home_latlon:
                    center = user_home_latlon[u]
                else:
                    lat_i, lon_i = item_latlon[pos_i]
                    if not (math.isnan(lat_i) or math.isnan(lon_i)):
                        center = (lat_i, lon_i)

                # If we have a center, attempt radius-aware sampling
                if center:
                    latc, lonc = center
                    candidates = [k for k in range(num_items) if k not in user_pos_map[u]]
                    random.shuffle(candidates)
                    for k in candidates:
                        latk, lonk = item_latlon[k]
                        # skip if no coordinates
                        if math.isnan(latk) or math.isnan(lonk):
                            continue
                        if haversine((latc, lonc), (latk, lonk)) <= radius_km:
                            neg_j = k
                            break
                # Fallback to uniform negative sampling
                if neg_j is None:
                    while True:
                        k = random.randrange(num_items)
                        if k not in user_pos_map[u]:
                            neg_j = k
                            break
            else:
                # Uniform negative sampling (without geo info for now)
                while True:
                    k = random.randrange(num_items)
                    if k not in user_pos_map[u]:
                        neg_j = k
                        break

            uu.append(u)
            ii.append(pos_i)
            jj.append(neg_j)

        yield (
            torch.tensor(uu, device=DEVICE),
            torch.tensor(ii, device=DEVICE),
            torch.tensor(jj, device=DEVICE)
        )

# ---------- Ranking Metrics Helper Functions ----------
def recall_at_k(ranked_items, ground_truth, k=10):
    hits = sum([1 for x in ranked_items[:k] if x in ground_truth])
    return hits / float(min(k, len(ground_truth))) if ground_truth else 0.0

def ndcg_at_k(ranked_items, ground_truth, k=10):
    dcg = sum([1.0 / math.log2(idx + 2) for idx, it in enumerate(ranked_items[:k]) if it in ground_truth])
    idcg = sum([1.0 / math.log2(i + 2) for i in range(min(k, len(ground_truth)))])
    return dcg / idcg if idcg > 0 else 0.0

def geo_discount(distance_km, R=5.0):
    return math.exp(-distance_km / R)

def geo_ndcg_at_k(ranked_items, ground_truth, user_loc, item_latlon, k=10, R=5.0):
    dcg = 0.0
    for idx, it in enumerate(ranked_items[:k], start=1):
        if it in ground_truth:
            w = 1.0
            if user_loc and not any(np.isnan(user_loc)):
                latu, lonu = user_loc
                lati, loni = item_latlon[it]
                if not math.isnan(lati) and not math.isnan(loni):
                    d = haversine((latu, lonu), (lati, loni))
                    w = geo_discount(d, R=R)
            dcg += w / math.log2(idx + 1)
    idcg = sum([1.0 / math.log2(i + 2) for i in range(min(k, len(ground_truth)))])
    return dcg / idcg if idcg > 0 else 0.0

def mrr(ranked_items, ground_truth, k=10):
    for idx, it in enumerate(ranked_items[:k], start=1):
        if it in ground_truth:
            return 1.0 / idx
    return 0.0

def rmse(preds, trues):
    return float(np.sqrt(np.mean((np.array(preds) - np.array(trues)) ** 2)))

def mae(preds, trues):
    return float(np.mean(np.abs(np.array(preds) - np.array(trues))))

# ---------- Dataset Helpers ----------
# For now, items have dummy coordinates → all NaNs
item_latlon = [(float(v[0]), float(v[1])) for v in data["item"].x[:, :2].tolist()]
user_home = {}  # optional; keep empty unless you have user locations


Baseline Model: LightGCN

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ---------- Basic LightGCN implementation ----------
class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, emb_dim=64, num_layers=4, alpha=None):
        super().__init__()
        self.num_users, self.num_items = num_users, num_items
        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)
        nn.init.normal_(self.user_emb.weight, std=0.1)
        nn.init.normal_(self.item_emb.weight, std=0.1)
        self.num_layers = num_layers
        self.alpha = alpha if alpha is not None else [1/(num_layers+1)]*(num_layers+1)

        # Check if the graph has user-item edges
        if ("user", "rates", "item") in data.edge_types:
            edge = data["user", "rates", "item"].edge_index
            u, i = edge[0], edge[1]
            # compute degrees
            deg_u = torch.bincount(u, minlength=num_users).float()
            deg_i = torch.bincount(i, minlength=num_items).float()
        else:
            # if no edges (edge_index empty)
            u = torch.tensor([], dtype=torch.long)
            i = torch.tensor([], dtype=torch.long)
            deg_u = torch.ones(num_users)
            deg_i = torch.ones(num_items)

        self.pairs = (u.to(DEVICE), i.to(DEVICE), deg_u.to(DEVICE), deg_i.to(DEVICE))

    def propagate(self, user_x, item_x):
        u, i, deg_u, deg_i = self.pairs
        all_user = [user_x]
        all_item = [item_x]

        for _ in range(self.num_layers):
            msg_u = torch.zeros_like(user_x)
            msg_i = torch.zeros_like(item_x)

            if len(u) > 0:  # only if edges exist
                msg_u.index_add_(
                    0,
                    u,
                    item_x[i] / torch.sqrt(deg_u[u].unsqueeze(1) * deg_i[i].unsqueeze(1) + 1e-8),
                )
                msg_i.index_add_(
                    0,
                    i,
                    user_x[u] / torch.sqrt(deg_i[i].unsqueeze(1) * deg_u[u].unsqueeze(1) + 1e-8),
                )

            user_x, item_x = msg_u, msg_i
            all_user.append(user_x)
            all_item.append(item_x)

        # Weighted layer-wise average
        alpha_tensor = torch.tensor(self.alpha, device=user_x.device).view(-1, 1, 1)
        user_out = (alpha_tensor * torch.stack(all_user)).sum(0)
        item_out = (alpha_tensor * torch.stack(all_item)).sum(0)
        return user_out, item_out

    def forward(self):
        u0 = self.user_emb.weight
        i0 = self.item_emb.weight
        return self.propagate(u0, i0)

    def score(self, users, items, user_emb=None, item_emb=None):
        if user_emb is None or item_emb is None:
            user_emb, item_emb = self.forward()
        return (user_emb[users] * item_emb[items]).sum(dim=1)


# ---------- BPR Loss Function ----------
def bpr_loss(pos_scores, neg_scores, reg=None, params: list = []):
    loss = -F.logsigmoid(pos_scores - neg_scores).mean()
    if reg:
        loss = loss + reg * sum(p.norm(2).pow(2) for p in params) / len(params)
    return loss


# ---------- Training Loop ----------
def train_lightgcn(epochs=5, emb_dim=64, batch_size=2048, lr=1e-3, reg=1e-4):
    model = LightGCN(num_users, num_items, emb_dim=emb_dim).to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    sampler = bpr_triplet_sampler(train_df, num_items, batch_size=batch_size)
    user_pos = {u: set(g["i"].values.tolist()) for u, g in train_df.groupby("u")}

    for ep in range(1, epochs + 1):
        model.train()
        total = 0.0
        steps = max(1, len(train_df) // batch_size)
        for step in range(steps):
            u, i, j = next(sampler)
            user_emb, item_emb = model()
            pos = model.score(u, i, user_emb, item_emb)
            neg = model.score(u, j, user_emb, item_emb)
            loss = bpr_loss(pos, neg, reg, [model.user_emb.weight, model.item_emb.weight])

            opt.zero_grad()
            loss.backward()
            opt.step()
            total += loss.detach().item()
        print(f"[LightGCN] Epoch {ep:02d} | Loss = {total/steps:.4f}")

    return model


# ---------- Run Training ----------
# lightgcn_model = train_lightgcn(epochs=30, emb_dim=128, batch_size=1024, lr=5e-4, reg=1e-4)

[LightGCN] Epoch 01 | Loss = 3.2987


KeyboardInterrupt: 

In [17]:
import math, copy, torch
import numpy as np

# ---------- Basic LightGCN implementation (+ early stopping) ----------
def train_lightgcn_with_eval(
    epochs=30,
    eval_every=5,
    emb_dim=128,
    num_layers=3,
    batch_size=1024,
    lr=5e-4,
    reg=1e-4,
    k_list=[5, 10, 20],
    early_stop_patience=5,      # epochs without improvement on NDCG@10
):
    model = LightGCN(num_users, num_items, emb_dim=emb_dim, num_layers=num_layers).to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    sampler = bpr_triplet_sampler(train_df, num_items, batch_size=batch_size)

    history = {"epoch": [], "loss": [], "val": []}
    best_metric = -1.0
    best_state = copy.deepcopy(model.state_dict())
    no_improve = 0

    steps_per_epoch = max(1, len(train_df) // batch_size)

    for ep in range(1, epochs + 1):
        model.train()
        running = 0.0

        for _ in range(steps_per_epoch):
            u, i, j = next(sampler)
            user_emb, item_emb = model()
            pos = model.score(u, i, user_emb, item_emb)
            neg = model.score(u, j, user_emb, item_emb)
            loss = bpr_loss(pos, neg, reg, [model.user_emb.weight, model.item_emb.weight])

            opt.zero_grad()
            loss.backward()
            opt.step()

            running += loss.detach().item()   # <- no warning

        avg_loss = running / steps_per_epoch
        history["epoch"].append(ep)
        history["loss"].append(avg_loss)

        print(f"[LightGCN] Epoch {ep:02d} | Loss = {avg_loss:.4f}")

        # ---- periodic validation for early stopping ----
        if (ep % eval_every == 0) or (ep == 1) or (ep == epochs):
            val_results = evaluate_model(model, train_df, val_df, k_list=k_list)
            history["val"].append((ep, val_results))
            metric = val_results.get("NDCG@10", 0.0)  # select your key metric

            pretty = " | ".join([f"{k}:{v:.4f}" for k, v in val_results.items()])
            print(f"Val @Epoch {ep}: {pretty}")

            # early stopping on the chosen metric
            if metric > best_metric + 1e-6:
                best_metric = metric
                best_state = copy.deepcopy(model.state_dict())
                no_improve = 0
                print("New best model (by NDCG@10).")
            else:
                no_improve += 1
                if no_improve >= early_stop_patience:
                    print(f"Early stopping (no improvement for {early_stop_patience} evals).")
                    break

    # load best and report final val/test
    model.load_state_dict(best_state)
    print("\nLoaded best model (by NDCG@10).")

    val_results  = evaluate_model(model, train_df, val_df,  k_list=k_list)
    test_results = evaluate_model(model, train_df, test_df, k_list=k_list)

    print("\n✅ Final Validation:")
    for k, v in val_results.items():
        print(f"  {k}: {v:.4f}")

    print("\n✅ Final Test:")
    for k, v in test_results.items():
        print(f"  {k}: {v:.4f}")

    return model, {"history": history, "val": val_results, "test": test_results}

# ---- Run it ----
lightgcn_model, eval_summary = train_lightgcn_with_eval(
    epochs=30,            # increase if still improving
    eval_every=5,         # validate every 5 epochs
    emb_dim=128,          # try 32/64/128
    num_layers=3,         # try 2–4
    batch_size=1024,      # adjust for memory
    lr=5e-4,
    reg=1e-4,
    k_list=[5, 10, 20],
    early_stop_patience=4
)

[LightGCN] Epoch 01 | Loss = 3.2955
Evaluating on 3628 users...


100%|██████████| 3628/3628 [00:02<00:00, 1609.76it/s]


Val @Epoch 1: Recall@5:0.0002 | NDCG@5:0.0002 | MRR@5:0.0003 | Recall@10:0.0006 | NDCG@10:0.0003 | MRR@10:0.0004 | Recall@20:0.0011 | NDCG@20:0.0005 | MRR@20:0.0005
New best model (by NDCG@10).
[LightGCN] Epoch 02 | Loss = 2.1546
[LightGCN] Epoch 03 | Loss = 1.5223
[LightGCN] Epoch 04 | Loss = 1.1664
[LightGCN] Epoch 05 | Loss = 0.9640
Evaluating on 3628 users...


100%|██████████| 3628/3628 [00:02<00:00, 1705.41it/s]


Val @Epoch 5: Recall@5:0.0012 | NDCG@5:0.0010 | MRR@5:0.0012 | Recall@10:0.0016 | NDCG@10:0.0011 | MRR@10:0.0013 | Recall@20:0.0021 | NDCG@20:0.0013 | MRR@20:0.0014
New best model (by NDCG@10).
[LightGCN] Epoch 06 | Loss = 0.8483
[LightGCN] Epoch 07 | Loss = 0.7820
[LightGCN] Epoch 08 | Loss = 0.7440
[LightGCN] Epoch 09 | Loss = 0.7222
[LightGCN] Epoch 10 | Loss = 0.7096
Evaluating on 3628 users...


...

Val @Epoch 30: Recall@5:0.0022 | NDCG@5:0.0022 | MRR@5:0.0024 | Recall@10:0.0042 | NDCG@10:0.0029 | MRR@10:0.0029 | Recall@20:0.0046 | NDCG@20:0.0031 | MRR@20:0.0030
New best model (by NDCG@10).

Loaded best model (by NDCG@10).
Evaluating on 3628 users...


100%|██████████| 3628/3628 [00:53<00:00, 68.32it/s]


Evaluating on 3642 users...


100%|██████████| 3642/3642 [00:52<00:00, 68.82it/s]


✅ Final Validation:
  Recall@5: 0.0022
  NDCG@5: 0.0022
  MRR@5: 0.0024
  Recall@10: 0.0042
  NDCG@10: 0.0029
  MRR@10: 0.0029
  Recall@20: 0.0046
  NDCG@20: 0.0031
  MRR@20: 0.0030

✅ Final Test:
  Recall@5: 0.0018
  NDCG@5: 0.0021
  MRR@5: 0.0029
  Recall@10: 0.0029
  NDCG@10: 0.0025
  MRR@10: 0.0033
  Recall@20: 0.0038
  NDCG@20: 0.0028
  MRR@20: 0.0034


In [14]:
import torch
import numpy as np
from tqdm import tqdm

# ---------- Eval helpers ----------
def evaluate_model(model, train_df, eval_df, k_list=[5, 10, 20]):
    """
    Evaluate a trained LightGCN model using Recall@K, NDCG@K, and MRR@K.
    """
    model.eval()
    with torch.no_grad():
        user_emb, item_emb = model.forward()

    # Build user->train_items map to exclude seen items from ranking
    train_user_pos = train_df.groupby("u")["i"].apply(set).to_dict()
    eval_user_pos = eval_df.groupby("u")["i"].apply(set).to_dict()

    recall_scores = {k: [] for k in k_list}
    ndcg_scores = {k: [] for k in k_list}
    mrr_scores = {k: [] for k in k_list}

    print(f"Evaluating on {len(eval_user_pos)} users...")
    for u, gt_items in tqdm(eval_user_pos.items()):
        if len(gt_items) == 0:
            continue

        # Compute scores for all items
        user_vec = user_emb[u].unsqueeze(0)
        scores = torch.matmul(user_vec, item_emb.T).squeeze(0)

        # Mask out items the user has already interacted with (training set)
        seen_items = train_user_pos.get(u, set())
        scores[list(seen_items)] = -1e9

        # Get top-K recommendations
        ranked_items = torch.topk(scores, k=max(k_list)).indices.cpu().numpy().tolist()

        # Compute metrics for each K
        for k in k_list:
            recall_scores[k].append(recall_at_k(ranked_items, gt_items, k))
            ndcg_scores[k].append(ndcg_at_k(ranked_items, gt_items, k))
            mrr_scores[k].append(mrr(ranked_items, gt_items, k))

    # Aggregate metrics
    results = {}
    for k in k_list:
        results[f"Recall@{k}"] = np.mean(recall_scores[k]) if recall_scores[k] else 0.0
        results[f"NDCG@{k}"] = np.mean(ndcg_scores[k]) if ndcg_scores[k] else 0.0
        results[f"MRR@{k}"] = np.mean(mrr_scores[k]) if mrr_scores[k] else 0.0

    return results

Evaluating on 3628 users...


100%|██████████| 3628/3628 [00:01<00:00, 2680.22it/s]


Evaluating on 3642 users...


100%|██████████| 3642/3642 [00:01<00:00, 2612.05it/s]


✅ Validation Results:
  Recall@5: 0.0021
  NDCG@5: 0.0020
  MRR@5: 0.0020
  Recall@10: 0.0024
  NDCG@10: 0.0021
  MRR@10: 0.0022
  Recall@20: 0.0029
  NDCG@20: 0.0023
  MRR@20: 0.0023

✅ Test Results:
  Recall@5: 0.0007
  NDCG@5: 0.0008
  MRR@5: 0.0012
  Recall@10: 0.0007
  NDCG@10: 0.0008
  MRR@10: 0.0012
  Recall@20: 0.0019
  NDCG@20: 0.0011
  MRR@20: 0.0013


In [ ]:
# ---------- Evaluate model ----------
val_results = evaluate_model(lightgcn_model, train_df, val_df, k_list=[5, 10, 20])
test_results = evaluate_model(lightgcn_model, train_df, test_df, k_list=[5, 10, 20])

print("\n✅ Validation Results:")
for k, v in val_results.items():
    print(f"  {k}: {v:.4f}")

print("\n✅ Test Results:")
for k, v in test_results.items():
    print(f"  {k}: {v:.4f}")

Extension Model 1: LightGCL

Elements:
1. Contrastive self-supervision

In [ ]:
def build_svd_view(train_df, num_users, num_items, rank=64):
    # Sparse user-item matrix
    rows = torch.tensor(train_df["u"].values, dtype=torch.long)
    cols = torch.tensor(train_df["i"].values, dtype=torch.long)
    vals = torch.ones(len(train_df), dtype=torch.float32)
    A = torch.sparse_coo_tensor(
        indices=torch.stack([rows, cols]), values=vals, size=(num_users, num_items)
    ).to_dense()  # for simplicity; if too big, sample or chunk
    U, S, Vt = torch.linalg.svd(A, full_matrices=False)
    Uk = U[:, :rank] * S[:rank]
    Vk = Vt[:rank, :].T * S[:rank]
    return Uk.to(DEVICE), Vk.to(DEVICE)

def info_nce(z, z_tgt, temperature=0.2):
    z = F.normalize(z, dim=1)
    z_tgt = F.normalize(z_tgt, dim=1)
    logits = z @ z_tgt.T / temperature
    labels = torch.arange(z.size(0), device=z.device)
    return F.cross_entropy(logits, labels)

class LightGCL(nn.Module):
    def __init__(self, base: LightGCN, lambda_cl=0.1, svd_rank=64):
        super().__init__()
        self.base = base
        self.lambda_cl = lambda_cl
        self.Uk, self.Vk = build_svd_view(train_df, base.num_users, base.num_items, rank=svd_rank)

    def training_step(self, batch, opt, reg=1e-4):
        u,i,j = batch
        user_emb, item_emb = self.base()
        pos = self.base.score(u,i,user_emb,item_emb)
        neg = self.base.score(u,j,user_emb,item_emb)
        loss_bpr = bpr_loss(pos,neg,reg,[self.base.user_emb.weight, self.base.item_emb.weight])
        # contrastive loss: align learned embeddings with SVD embeddings
        cl_u = info_nce(user_emb, self.Uk)
        cl_i = info_nce(item_emb, self.Vk)
        loss = loss_bpr + self.lambda_cl*(cl_u + cl_i)
        opt.zero_grad(); loss.backward(); opt.step()
        return float(loss.item())

def train_lightgcl(epochs=5, emb_dim=64, batch_size=2048, lr=1e-3, reg=1e-4, lambda_cl=0.1):
    base = LightGCN(num_users, num_items, emb_dim=emb_dim).to(DEVICE)
    model = LightGCL(base, lambda_cl=lambda_cl, svd_rank=emb_dim).to(DEVICE)
    opt = torch.optim.Adam(base.parameters(), lr=lr)
    sampler = bpr_triplet_sampler(train_df, num_items, batch_size=batch_size)
    for ep in range(1, epochs+1):
        total=0.0
        for step in range(max(1, len(train_df)//batch_size)):
            u,i,j = next(sampler)
            total += model.training_step((u,i,j), opt, reg=reg)
        print(f"[LightGCL] epoch {ep} loss {total/(step+1):.4f}")
    return model

lightgcl_model = train_lightgcl(epochs=5, emb_dim=64, lambda_cl=0.1)


Extension Model 2: LightGCL + Geographical Awareness

Elements:
1. Contrastive self-supervision
2. Distance-aware scoring
3. Radius-aware negative sampling

In [ ]:
class LightGCL_Geo(LightGCL):
    def __init__(self, base: LightGCN, beta=0.2, R=5.0, lambda_cl=0.1):
        super().__init__(base, lambda_cl=lambda_cl, svd_rank=base.user_emb.embedding_dim)
        self.beta = beta
        self.R = R

    def geo_term(self, users, items, user_home, item_latlon):
        vals = []
        for u,i in zip(users.tolist(), items.tolist()):
            uh = user_home.get(u)
            if uh is None:
                lati,loni = item_latlon[i]
                if math.isnan(lati) or math.isnan(loni):
                    vals.append(0.0); continue
                uh = (lati,loni)  # weak fallback
            lati,loni = item_latlon[i]
            if math.isnan(lati) or math.isnan(loni):
                vals.append(0.0); continue
            d = haversine(uh, (lati,loni))
            vals.append(math.exp(-d/self.R))
        return torch.tensor(vals, device=DEVICE, dtype=torch.float32)

    def training_step(self, batch, opt, reg=1e-4):
        u,i,j = batch
        user_emb, item_emb = self.base()
        pos = self.base.score(u,i,user_emb,item_emb) + self.beta * self.geo_term(u,i,user_home,item_latlon)
        neg = self.base.score(u,j,user_emb,item_emb) + self.beta * self.geo_term(u,j,user_home,item_latlon)
        loss_bpr = bpr_loss(pos,neg,reg,[self.base.user_emb.weight, self.base.item_emb.weight])
        cl_u = info_nce(user_emb, self.Uk)
        cl_i = info_nce(item_emb, self.Vk)
        loss = loss_bpr + self.lambda_cl*(cl_u + cl_i)
        opt.zero_grad(); loss.backward(); opt.step()
        return float(loss.item())

def train_lightgcl_geo(epochs=5, emb_dim=64, batch_size=2048, lr=1e-3, reg=1e-4, lambda_cl=0.1, beta=0.2, R=5.0):
    base = LightGCN(num_users, num_items, emb_dim=emb_dim).to(DEVICE)
    model = LightGCL_Geo(base, beta=beta, R=R, lambda_cl=lambda_cl).to(DEVICE)
    opt = torch.optim.Adam(base.parameters(), lr=lr)
    sampler = bpr_triplet_sampler(train_df, num_items, radius_km=R,
                                  item_latlon=item_latlon, user_pos_map=None,
                                  user_home_latlon=user_home, batch_size=batch_size)
    for ep in range(1, epochs+1):
        total=0.0
        for step in range(max(1, len(train_df)//batch_size)):
            u,i,j = next(sampler)
            total += model.training_step((u,i,j), opt, reg=reg)
        print(f"[LightGCL+Geo] epoch {ep} loss {total/(step+1):.4f}")
    return model

lightgcl_geo_model = train_lightgcl_geo(epochs=5, emb_dim=64, beta=0.3, R=5.0)


Comparative Model 1: PinSage

In [ ]:
class PinSageItemEncoder(nn.Module):
    def __init__(self, in_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.proj = nn.Linear(in_dim, hidden_dim) if in_dim>0 else None
        self.layers = nn.ModuleList([
            PinSAGEConv(in_channels=hidden_dim, out_channels=hidden_dim, heads=1)
            for _ in range(num_layers)
        ])

    def forward(self, x, edge_index):
        h = x
        if self.proj is not None:
            h = self.proj(h)
        for conv in self.layers:
            h = conv(h, edge_index)
            h = F.relu(h)
        return F.normalize(h, dim=1)

class PinSageRecommender(nn.Module):
    def __init__(self, item_in_dim, hidden_dim=64):
        super().__init__()
        self.item_enc = PinSAGEItemEncoder(item_in_dim, hidden_dim=hidden_dim)

    def forward(self, item_x, ii_edge):
        return self.item_enc(item_x, ii_edge)

    def user_repr(self, user_pos_items, item_emb):
        # mean pool over positives
        return F.normalize(item_emb[user_pos_items].mean(0, keepdim=True), dim=1)

def train_pinsage(epochs=5, hidden_dim=64, batch_size=2048, lr=1e-3):
    item_x = data['item'].x
    ii = data['item','similar','item'].edge_index
    model = PinSageRecommender(item_x.size(1), hidden_dim=hidden_dim).to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    user_pos_map = {u:set(g["i"].values.tolist()) for u,g in train_df.groupby("u")}
    sampler = bpr_triplet_sampler(train_df, num_items, batch_size=batch_size)
    for ep in range(1, epochs+1):
        total = 0.0
        for step in range(max(1, len(train_df)//batch_size)):
            u, i, j = next(sampler)
            item_emb = model(item_x, ii)
            u_repr = item_emb[i]  # treat pos item as anchor (approximate)
            pos = (u_repr * item_emb[i]).sum(1)
            neg = (u_repr * item_emb[j]).sum(1)
            loss = -F.logsigmoid(pos - neg).mean()
            opt.zero_grad(); loss.backward(); opt.step()
            total += float(loss)
        print(f"[PinSage] epoch {ep} loss {total/(step+1):.4f}")
    return model

pinsage_model = train_pinsage(epochs=5, hidden_dim=64)


Comparative Model 2: LightGCN + HGT

Elements:
1. Base Model LightGCN
2. HGT item representation layer

In [ ]:
# Build minimal hetero metadata graph for items: (item)-[has_cat]->(cat), (item)-[has_price]->(price_level)
# From df categories/price
cat_encoder = LabelEncoder()
all_cats = []
for row in df["categories"]:
    if isinstance(row, list):
        all_cats += row
    elif pd.notna(row):
        all_cats.append(str(row))
if len(all_cats)==0:
    all_cats = ["unknown"]
cat_encoder.fit(list(set(all_cats)))
num_cats = len(cat_encoder.classes_)
price_levels = sorted(list(set([int(p) if pd.notna(p) else 0 for p in df["price"]])))
price_to_idx = {p:i for i,p in enumerate(price_levels)}
num_prices = len(price_levels)

meta = HeteroData()
meta["item"].num_nodes = num_items
meta["cat"].num_nodes = num_cats
meta["price"].num_nodes = num_prices
# item->cat edges
src, dst = [], []
for i, g in df.groupby("i"):
    cats = g["categories"].iloc[0]
    if isinstance(cats, list) and len(cats)>0:
        for c in cats[:3]:
            src.append(i); dst.append(cat_encoder.transform([c])[0])
    else:
        src.append(i); dst.append(cat_encoder.transform([cat_encoder.classes_[0]])[0])
meta["item","has_cat","cat"].edge_index = torch.tensor([src,dst], dtype=torch.long)
# item->price edges
src, dst = [], []
for i, g in df.groupby("i"):
    p = g["price"].iloc[0]
    p = int(p) if pd.notna(p) else 0
    src.append(i); dst.append(price_to_idx.get(p, 0))
meta["item","has_price","price"].edge_index = torch.tensor([src,dst], dtype=torch.long)
meta = meta.to(DEVICE)

class HGTItemEncoder(nn.Module):
    def __init__(self, hidden=64, heads=2, layers=2):
        super().__init__()
        self.emb = nn.ModuleDict({
            'item': nn.Embedding(num_items, hidden),
            'cat': nn.Embedding(num_cats, hidden),
            'price': nn.Embedding(num_prices, hidden),
        })
        for k in self.emb:
            nn.init.xavier_uniform_(self.emb[k].weight)
        self.layers = nn.ModuleList([
            HGTConv(hidden_channels=hidden, out_channels=hidden, num_types=3, num_relations=2, heads=heads)
            for _ in range(layers)
        ])

    def forward(self, meta: HeteroData):
        x_dict = {k: self.emb[k].weight for k in ['item','cat','price']}
        for conv in self.layers:
            x_dict = conv(x_dict, meta.edge_index_dict)
            x_dict = {k: F.relu(v) for k,v in x_dict.items()}
        return x_dict['item']

class LightGCN_HGT(nn.Module):
    def __init__(self, lightgcn: LightGCN, hgt_hidden=64):
        super().__init__()
        self.lgcn = lightgcn
        self.hgt = HGTItemEncoder(hidden=hgt_hidden)
        self.fuse = nn.Linear(self.lgcn.user_emb.embedding_dim + hgt_hidden, self.lgcn.user_emb.embedding_dim)

    def fused_item_emb(self):
        user_e, item_e = self.lgcn()
        hgt_item = self.hgt(meta)
        item_fused = self.fuse(torch.cat([item_e, hgt_item], dim=1))
        return user_e, item_fused

    def score(self, users, items):
        ue, ie = self.fused_item_emb()
        return (ue[users]*ie[items]).sum(1)

def train_lightgcn_hgt(epochs=5, emb_dim=64, batch_size=2048, lr=1e-3, reg=1e-4):
    base = LightGCN(num_users, num_items, emb_dim=emb_dim).to(DEVICE)
    model = LightGCN_HGT(base, hgt_hidden=emb_dim).to(DEVICE)
    params = list(model.lgcn.parameters()) + list(model.hgt.parameters()) + list(model.fuse.parameters())
    opt = torch.optim.Adam(params, lr=lr)
    sampler = bpr_triplet_sampler(train_df, num_items, batch_size=batch_size)
    for ep in range(1, epochs+1):
        total=0.0
        for step in range(max(1, len(train_df)//batch_size)):
            u,i,j = next(sampler)
            ue, ie = model.lgcn()
            hgt_item = model.hgt(meta)
            fused = model.fuse(torch.cat([ie, hgt_item], dim=1))
            pos = (ue[u]*fused[i]).sum(1)
            neg = (ue[u]*fused[j]).sum(1)
            loss = bpr_loss(pos,neg,reg, params)
            opt.zero_grad(); loss.backward(); opt.step()
            total += float(loss)
        print(f"[LightGCN+HGT] epoch {ep} loss {total/(step+1):.4f}")
    return model

lghgt_model = train_lightgcn_hgt(epochs=5, emb_dim=64)


Evaluation Helpers

In [ ]:
def full_ranking_scores(model, model_name, K_list=(5,10), geo_R=5.0, geo_beta=0.0):
    # prepare user->train positives and heldouts
    train_pos = {u:set(g["i"].values.tolist()) for u,g in train_df.groupby("u")}
    gt_val = {u:set(g["i"].values.tolist()) for u,g in val_df.groupby("u")}
    gt_test = {u:set(g["i"].values.tolist()) for u,g in test_df.groupby("u")}
    # compute item/user embeddings or scorers per model
    with torch.no_grad():
        if isinstance(model, LightGCN):
            ue, ie = model()
            def score_u(u):
                # mask train positives for ranking
                s = (ue[u].unsqueeze(0) * ie).sum(1)
                for it in train_pos.get(u,[]):
                    s[it] = -1e9
                return s
        elif isinstance(model, LightGCL) and not isinstance(model, LightGCL_Geo):
            ue, ie = model.base()
            def score_u(u):
                s = (ue[u].unsqueeze(0) * ie).sum(1)
                for it in train_pos.get(u,[]): s[it] = -1e9
                return s
        elif isinstance(model, LightGCL_Geo):
            ue, ie = model.base()
            def score_u(u):
                s = (ue[u].unsqueeze(0) * ie).sum(1)
                # add geo distance term
                geo = []
                for it in range(num_items):
                    uh = user_home.get(u)
                    lati,loni = item_latlon[it]
                    w = 0.0
                    if uh and not any(np.isnan(uh)) and not math.isnan(lati) and not math.isnan(loni):
                        d = haversine(uh,(lati,loni))
                        w = math.exp(-d/model.R)
                    geo.append(w)
                s = s + model.beta*torch.tensor(geo, device=DEVICE)
                for it in train_pos.get(u,[]): s[it] = -1e9
                return s
        elif isinstance(model, PinSageRecommender):
            item_x = data['item'].x
            ii = data['item','similar','item'].edge_index
            item_emb = model(item_x, ii)
            def score_u(u):
                pos = list(train_pos.get(u, [])) or [0]
                uvec = item_emb[pos].mean(0, keepdim=True)
                s = (uvec * item_emb).sum(1)
                for it in train_pos.get(u,[]): s[it] = -1e9
                return s
        elif isinstance(model, LightGCN_HGT):
            def score_u(u):
                ue, ie = model.fused_item_emb()
                s = (ue[u].unsqueeze(0) * ie).sum(1)
                for it in train_pos.get(u,[]): s[it] = -1e9
                return s
        else:
            raise ValueError("Unknown model for evaluation")

        def evaluate(gt_dict, tag):
            rec, ndcg, gndcg, rr = [], [], [], []
            for u in gt_dict.keys():
                s = score_u(u)
                ranked = torch.topk(s, k=min(100, num_items)).indices.tolist()
                truth = gt_dict[u]
                rec.append(recall_at_k(ranked, truth, k=max(K_list)))
                ndcg.append(ndcg_at_k(ranked, truth, k=max(K_list)))
                # geo-aware NDCG
                u_loc = user_home.get(u)
                gndcg.append(geo_ndcg_at_k(ranked, truth, u_loc, item_latlon, k=max(K_list), R=geo_R))
                rr.append(mrr(ranked, truth, k=max(K_list)))
            return np.mean(rec), np.mean(ndcg), np.mean(gndcg), np.mean(rr)

        val_scores = evaluate(gt_val, "val")
        test_scores = evaluate(gt_test, "test")
        print(f"[{model_name}] Val  Recall@K:{val_scores[0]:.4f}  NDCG@K:{val_scores[1]:.4f}  GeoNDCG@K:{val_scores[2]:.4f}  MRR:{val_scores[3]:.4f}")
        print(f"[{model_name}] Test Recall@K:{test_scores[0]:.4f}  NDCG@K:{test_scores[1]:.4f}  GeoNDCG@K:{test_scores[2]:.4f}  MRR:{test_scores[3]:.4f}")
        return {"model": model_name, "val_recall":val_scores[0], "val_ndcg":val_scores[1], "val_geondcg":val_scores[2], "val_mrr":val_scores[3],
                "test_recall":test_scores[0], "test_ndcg":test_scores[1], "test_geondcg":test_scores[2], "test_mrr":test_scores[3]}

results = []
results.append(full_ranking_scores(lightgcn_model, "LightGCN"))
results.append(full_ranking_scores(lightgcl_model.base, "LightGCL (base view)"))
results.append(full_ranking_scores(lightgcl_geo_model.base, "LightGCL+Geo (inference uses geo)", geo_R=5.0))
results.append(full_ranking_scores(pinsage_model, "PinSage"))
results.append(full_ranking_scores(lghgt_model.lgcn, "LightGCN+HGT (fused)"))
pd.DataFrame(results)


Rating Metrics (RMSE/MAE)

In [ ]:
def pointwise_eval(model, which="val"):
    df_eval = val_df if which=="val" else test_df
    preds, trues = [], []
    with torch.no_grad():
        if isinstance(model, LightGCN):
            ue, ie = model()
            for _, r in df_eval.iterrows():
                preds.append(float((ue[r.u] * ie[r.i]).sum().item()))
                trues.append(float(r.rating))
        elif isinstance(model, LightGCL):
            ue, ie = model.base()
            for _, r in df_eval.iterrows():
                s = float((ue[r.u]*ie[r.i]).sum().item())
                preds.append(s); trues.append(float(r.rating))
        elif isinstance(model, LightGCL_Geo):
            ue, ie = model.base()
            for _, r in df_eval.iterrows():
                s = float((ue[r.u]*ie[r.i]).sum().item())
                # add geo term
                uh = user_home.get(int(r.u))
                lati,loni = item_latlon[int(r.i)]
                if uh and not any(np.isnan(uh)) and not math.isnan(lati) and not math.isnan(loni):
                    d = haversine(uh,(lati,loni))
                    s += model.beta*math.exp(-d/model.R)
                preds.append(s); trues.append(float(r.rating))
        elif isinstance(model, PinSageRecommender):
            item_x = data['item'].x
            ii = data['item','similar','item'].edge_index
            item_emb = model(item_x, ii)
            for _, r in df_eval.iterrows():
                # approximate user vector by mean of their train items
                pos = train_df[train_df.u==r.u]["i"].values
                if len(pos)==0: continue
                uvec = item_emb[pos].mean(0)
                preds.append(float((uvec * item_emb[r.i]).sum().item()))
                trues.append(float(r.rating))
        elif isinstance(model, LightGCN_HGT):
            ue, ie = model.fused_item_emb()
            for _, r in df_eval.iterrows():
                preds.append(float((ue[r.u]*ie[r.i]).sum().item()))
                trues.append(float(r.rating))
        else:
            raise ValueError("unknown model")
    return {"rmse": rmse(preds,trues), "mae": mae(preds,trues), "n": len(trues)}

for name, mdl in [
    ("LightGCN", lightgcn_model),
    ("LightGCL", lightgcl_model),
    ("LightGCL+Geo", lightgcl_geo_model),
    ("PinSage", pinsage_model),
    ("LightGCN+HGT", lghgt_model),
]:
    pe = pointwise_eval(mdl, "test")
    print(f"{name} rating RMSE={pe['rmse']:.4f} MAE={pe['mae']:.4f} (n={pe['n']})")


Vistualization: Comparison Table

In [ ]:
summary = pd.DataFrame(results).assign(
    rmse=[pointwise_eval(m, 'test')["rmse"] for m in [lightgcn_model, lightgcl_model, lightgcl_geo_model, pinsage_model, lghgt_model]],
    mae =[pointwise_eval(m, 'test')["mae"]  for m in [lightgcn_model, lightgcl_model, lightgcl_geo_model, pinsage_model, lghgt_model]],
)
summary = summary[["model","test_recall","test_ndcg","test_geondcg","test_mrr","rmse","mae"]]
summary.sort_values("test_ndcg", ascending=False)